In [ ]:
SCENE = "mic"
VER = "1027"
DEFORM = f"{SCENE}"
eval_only = True

In [ ]:
# # DEBUG ONLY, MUST BE REMOVED
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Init

In [ ]:
while True:
    try:
        already_run_flag
    except:
        break
    assert False, "Run twice"

already_run_flag = True

In [ ]:
cd ProjectNerf

In [ ]:
!nvidia-smi

In [ ]:
import logging
from tqdm import tqdm
from tqdm.contrib.logging import logging_redirect_tqdm
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    '%(asctime)s - %(levelname)s:\t%(message)s',
    datefmt='%Y-%m-%d %H:%M:%S')

fh = logging.FileHandler(f"scratch/train_field.log")
fh.setLevel(logging.INFO)
fh.setFormatter(formatter)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)

logger.addHandler(ch)
logger.addHandler(fh)
logging.info("Init")

In [ ]:
!git pull

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import tqdm
import random
import open3d as o3d
from tqdm import tqdm

In [ ]:
from torchvision.transforms import transforms
import cv2
import cv2 as cv
from PIL import Image
import scipy
import scipy.linalg
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
cd cppext

In [ ]:
!python setup.py install

In [ ]:
cd ..

# Read Data

In [ ]:
DATASET = f"./datasets/Scratch/ns_{SCENE}"
CLOUD_NAME = "cloud_pn"
# CLOUD_NAME = "cloud_pn_aftergrow"
# CLOUD_NAME = "cloud_gt"
LIGHT_IND = 3
CLOUD_SCALE = 1 / 1
IMAGE_SCALE = 1
IMAGE_SCALE_TRAIN = 1 #7
IMAGE_PATCH_TRAIN = 1
TRAIN_RANDOM_AUG = False # True
TRAIN_REPEAT = 1
NUM_PTS = 2**22 if CLOUD_NAME == "cloud_gt" else 2**21
BORDER_FIX_KER = 0
BORDER_FIX_THRES = 8
DEFAULT_POV = 0
EVAL_MASK_ONLY = False
BG_COLOR = [0, 0, 0]
IMAGE_TRAIN_BG_PAD = 6
NORM_MESH_PARAM = o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=512)


import json
train_config = json.load(open(f"{DATASET}/transforms_train.json"))
grow_extra_config = None #json.load(open(f"{DATASET}/transforms_val.json"))
prune_extra_config = json.load(open(f"{DATASET}/transforms_val.json"))
# valid_config = json.load(open(f"{DATASET}/transforms_val.json"))
valid_config = json.load(open(f"{DATASET}/transforms_test.json"))

DEFAULT_VIEW_DIR = [-1., -1., -1.]

TRI_EXTEND = False

IGNORE_CONF = False if "aftergrow" not in CLOUD_NAME else True #True # if CLOUD_NAME == "cloud_gt" else False
IGNORE_EMBED = True if CLOUD_NAME == "cloud_gt" else False


## Read Cloud

In [ ]:
pth = torch.load(f"{DATASET}/{CLOUD_NAME}.pth")
pth.keys()

In [ ]:
pts = pth['points_with_color'].float().cpu()
pts_conf = pth['confidence'].float().cpu()
pts_embed = pth['feature'].float().cpu()
pts.shape, pts_conf.shape, pts_embed.shape

In [ ]:
def extend_pts(pts, N, features=[]):
    import scipy
    import scipy.spatial
    tri = torch.from_numpy(scipy.spatial.Delaunay(pts.numpy()).simplices)
    pair = []
    for i in range(4):
        for j in range(i):
            pair.append(tri[:, [i, j]])

    pair = torch.cat(pair, dim=0).long()

    dist = (pts[pair[:, 0]] - pts[pair[:, 1]]).norm(dim=-1)
    mean = dist.mean()
    std = dist.std()
    mask = dist < mean + 2 * std
    pair = pair[mask]

    cho = torch.randint(low=0, high=pair.shape[0], size=[N - pts.shape[0]])
    coef = torch.rand([N - pts.shape[0], 1])
    a = pair[cho, 0]
    b = pair[cho, 1]

    new_pts = pts[a] * coef + pts[b] * (1 - coef)
    new_pts = torch.cat([pts, new_pts], dim=0)

    new_features = []
    for f in features:
        f = f.cpu()
        c = coef if len(f.shape) == 2 else coef.squeeze()
        new_f = f[a] * c + f[b] * (1 - c)
        new_f = torch.cat([f, new_f], dim=0)
        new_features.append(new_f)

    if len(new_features) > 0:
        return new_pts, new_features
    return new_pts


if TRI_EXTEND and pts.shape[0] < NUM_PTS:
    pts, (pts_col, pts_conf, pts_embed) = extend_pts(pts[:, :3], NUM_PTS, (pts[:, 3:], pts_conf, pts_embed))
    pts = torch.cat([pts, pts_col], dim=-1)

pts.shape, pts_conf.shape, pts_embed.shape

In [ ]:
if IGNORE_CONF:
    print("ignore conf")
    pts_conf = pts_conf * 0 + 0.5

if IGNORE_EMBED:
    print("ignore embed")
    pts_embed = torch.randn_like(pts_embed)

In [ ]:
mask = torch.randperm(pts.shape[0])[:NUM_PTS]

pts, pts_col = pts[mask].split([3, 3], dim=-1)
pts *= CLOUD_SCALE
if pts_col.max() >= 10:
    print("Convert color")
    pts_col = (pts_col / 255).clamp(min=0, max=1)
pts_conf = pts_conf[mask].clamp(min=0, max=1)
pts_embed = pts_embed[mask]
# pts_embed = torch.cat([pts_col, pts_embed], dim=-1)

pts.shape, pts_col.aminmax(), pts_conf.aminmax()

In [ ]:
# mov = (pts.max() - pts.min()) / pts.shape[0]**0.5
# pts += mov * torch.randn_like(pts)
# pts.shape, pts_col.shape, pts_conf.shape, pts_embed.shape

In [ ]:
def intersect_box(line_o, line_d, box_d, box_u, fetch_range=False, positive_only=True):
    import math
    inv_d = 1 / line_d
    A = (box_d - line_o) * inv_d
    B = (box_u - line_o) * inv_d

    def fmx(x):
        x = x.clone()
        x[x.isnan()] = -math.inf
        return x
    def fmn(x):
        x = x.clone()
        x[x.isnan()] = math.inf
        return x

    def pwmin(A, B):
        x = torch.minimum(fmn(A), fmn(B))
        x[A.isnan() & B.isnan()] = math.nan
        return x
    def pwmax(A, B):
        x = torch.maximum(fmx(A), fmx(B))
        x[A.isnan() & B.isnan()] = math.nan
        return x


    pmin = pwmin(A, B)
    pmax = pwmax(A, B)

    vmin = fmx(pmin).max(dim=-1).values
    vmax = fmn(pmax).min(dim=-1).values

    if positive_only:
        vmin = vmin.clamp(min=0)

    intersect = vmin + 1e-6 < vmax

    if fetch_range:
        return intersect, vmin, vmax

    return intersect

In [ ]:
def show_cloud(pts, pts_col=pts_col, show_norm=False):
    import open3d as o3d
    cloud = o3d.geometry.PointCloud()
    cloud.points = o3d.utility.Vector3dVector(pts.cpu().numpy())
    cloud.colors = o3d.utility.Vector3dVector(pts_col.cpu().numpy())
    cloud.estimate_normals()
    o3d.visualization.draw_geometries([cloud], point_show_normal=show_norm)

def estimate_norm(pts, pts_col=pts_col, param=None):
    import open3d as o3d
    cloud = o3d.geometry.PointCloud()
    cloud.points = o3d.utility.Vector3dVector(pts.cpu().numpy())
    cloud.colors = o3d.utility.Vector3dVector(pts_col.cpu().numpy())
    if param is None:
        cloud.estimate_normals()
    else:
        cloud.estimate_normals(search_param=param)

    return torch.from_numpy(np.asarray(cloud.normals))


In [ ]:
%matplotlib inline
def plot(pts, conf=None, rgb=None, pov=[DEFAULT_POV], revcol=[False], dpi=64, save=None):
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    import matplotlib.gridspec as gridspec
    import numpy as np

    if len(pts.shape) > 2:
        pts = pts.squeeze(0)
        if conf is not None:
            conf = conf.squeeze(0)
        if rgb is not None:
            rgb = rgb.squeeze(0)
    if conf is None:
        conf = torch.ones_like(pts[:, 0])
    if rgb is None:
        rgb = torch.zeros_like(pts)

    assert len(pts.shape) == 2

    pts = pts.clone().cpu().numpy()
    x, y, z = pts[:,0], pts[:,1], pts[:,2]
    x -= x.mean()
    y -= y.mean()
    z -= z.mean()

    for rc in revcol:
        rgba = torch.cat([rgb if not rc else 1 - rgb, conf.unsqueeze(-1)], dim=-1).cpu().numpy()

        def apply_i(pts, i):
            x, y, z = pts[...,0], pts[...,1], pts[...,2]
            if (i & 4) != 0:    x = -x
            if (i & 2) != 0:    y = -y
            if (i & 1) != 0:    z = -z
            return torch.stack([x, y, z], dim=-1)

        for i in pov:
            fig = plt.figure(dpi=dpi)
            fig.set_size_inches(12, 12, forward=False)
            # gs = gridspec.GridSpec(nrows=4, ncols=2, left=0.1, right=2.5, wspace=0.05, hspace=0.1, bottom=0.1, top=4.9)
            gs = gridspec.GridSpec(nrows=1, ncols=1, left=0, right=1, wspace=0, hspace=0, bottom=0.0, top=1)

            ax = fig.add_subplot(gs[0, 0], projection='3d')
            if not rc:
                pane_col = (0., 0., 0.5, 0.1)
            else:
                pane_col = (0., 0., 0., 0.95)
            ax.xaxis.set_pane_color(pane_col)
            ax.yaxis.set_pane_color(pane_col)
            ax.zaxis.set_pane_color(pane_col)

            x, y, z = pts[:,0], pts[:,1], pts[:,2]
            labx, laby, labz = 'x', 'y', 'z'
            if (i & 4) != 0:    x = -x; labx = '-x'
            if (i & 2) != 0:    y = -y; laby = '-y'
            if (i & 1) != 0:    z = -z; labz = '-z'

            ax.scatter(x, y, z, c=rgba, marker='.')

            xmid = (x.min() + x.max()) / 2
            ymid = (y.min() + y.max()) / 2
            zmid = (z.min() + z.max()) / 2
            gap = max(x.max() - x.min(), y.max() - y.min(), z.max() - z.min()) / 2

            ax.set_xlim(xmid - gap, xmid + gap)
            ax.set_ylim(ymid - gap, ymid + gap)
            ax.set_zlim(zmid - gap, zmid + gap)

            ax.set_xlabel(labx)
            ax.set_ylabel(laby)
            ax.set_zlabel(labz)

            if save is not None:
                plt.savefig(save)

            plt.show()


In [ ]:
mask = torch.randperm(pts.shape[0])[:524288]
plot(pts[mask], conf=pts_conf[mask], rgb=pts_col[mask],
     save=f"./scratch/ns_{SCENE}_{CLOUD_NAME}_cloud.png")

## Read Raw

In [ ]:
IMG_W = 800
IMG_H = 800

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 128
mpl.rcParams['savefig.pad_inches'] = 0

def show(img, save=None):
    # img = img.astype(np.int)
    fig = plt.figure()
    # fig.set_size_inches(height * img.shape[0] / img.shape[1], height, forward=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    # img = torch.from_numpy(cv2.resize(img.float().numpy(), [IMG_W, IMG_H]))
    if len(img.shape) == 3:
        ax.imshow(img)
    else:
        ax.imshow(img, cmap='gray')
    if save is not None:
        import imageio
        imageio.imwrite(save, img)
    plt.show()

def enhomo(x):
    return torch.cat([x, torch.ones(list(x.shape[:-1]) + [1], device=x.device, dtype=x.dtype)], dim=-1)

def dehomo(x):
    return x[..., :-1] / x[..., [-1]]


In [ ]:
def read_pfm(filename):
    import re
    file = open(filename, 'rb')
    header = file.readline().decode('utf-8').rstrip()
    if header == 'PF':
        color = True
    elif header == 'Pf':
        color = False
    else:
        raise Exception('Not a PFM file.')

    dim_match = re.match(r'^(\d+)\s(\d+)\s$', file.readline().decode('utf-8'))
    if dim_match:
        width, height = map(int, dim_match.groups())
    else:
        raise Exception('Malformed PFM header.')

    scale = float(file.readline().rstrip())
    if scale < 0:  # little-endian
        endian = '<'
        scale = -scale
    else:
        endian = '>'  # big-endian

    data = np.fromfile(file, endian + 'f')
    shape = (height, width, 3) if color else (height, width)

    data = np.reshape(data, shape)
    data = np.flipud(data)
    file.close()
    return data, scale

In [ ]:
def pix2pts_ns(cam, pix_ind, scale=1, rand=False):
    cam_mat_inv = cam[1]
    n, m = IMG_W * scale, IMG_H * scale
    tmp = torch.stack([pix_ind % m, pix_ind.div(m, rounding_mode='floor')], dim=-1).double()
    tmp = (tmp + 0.5).double()
    if rand:
        mov = (torch.randn_like(tmp) / 16).clamp(min=-1, max=1)
        tmp += mov
    tmp /= scale
    # print(tmp.shape)
    tmp = torch.cat([tmp, torch.ones_like(tmp)], dim=-1).double()
    # print(tmp.shape)
    tmp = dehomo(cam_mat_inv.matmul(tmp.unsqueeze(-1)).squeeze(-1))
    tmp = tmp * CLOUD_SCALE
    return tmp.float()

def pts2pix_ns(cam, pts):
    pts = pts / CLOUD_SCALE
    cam_mat = cam[0].to(pts.device)
    proj = dehomo(cam_mat.matmul(enhomo(pts).double().unsqueeze(-1)).squeeze(-1)).float()
    proj = torch.stack([proj[:, 1], proj[:, 0]], dim=-1)
    return proj.float()

def pix2ray_ns(cam, pix_ind, scale=1, rand=False):
    center = cam[2]
    pts = pix2pts_ns(cam, pix_ind, scale=scale, rand=rand)
    dir = (pts - center)
    dir /= dir.norm(dim=-1, keepdim=True).clamp(min=1e-7)
    return dir.float()

In [ ]:
def read_data_ns(config, i, for_eval=False, debug=False, ray_upsample=1, full_image=False, return_fn=True):
    import cv2
    import math

    img_file = f"{DATASET}/{config['frames'][i]['file_path']}.png"
    cam_ext = torch.tensor(config['frames'][i]['transform_matrix'], dtype=torch.double)
    cam_ext = cam_ext @ torch.tensor([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]).double()
    cam_ext = torch.tensor(scipy.linalg.inv(cam_ext)).double()

    focal = 0.5 * IMG_W / math.tan(0.5 * config['camera_angle_x'])
    cam_int = torch.tensor([[focal, 0, IMG_W / 2], [0, focal, IMG_H / 2], [0, 0, 1]], dtype=torch.double)

    raw_img = cv2.imread(img_file, cv2.IMREAD_UNCHANGED)
    opa = torch.from_numpy(raw_img[:, :, -1] / 255)
    img = torch.from_numpy(raw_img[:, :, :3][:, :, ::-1] / 255)

    # img2 = torch.from_numpy(cv2.imread(img_file)[:, :, ::-1] / 255)
    # show(img2)
    # show(img)
    # print((img2 - img).abs().max())
    # return

    opa_thres = opa[opa > 0].min() - 1e-6
    if debug:
        print("opa_thres", opa_thres)

    cam_mat = cam_int.double().mm(cam_ext.double()[:-1])
    cam_center = dehomo(torch.tensor(scipy.linalg.null_space(cam_mat.double())).squeeze()).float() * CLOUD_SCALE
    cam_mat_inv = torch.tensor(scipy.linalg.inv(torch.cat([cam_mat, torch.tensor([[0., 0., 0., 1.]])], dim=0).double()))
    cam = (cam_mat, cam_mat_inv, cam_center)

    orig_img = img
    orig_opa = opa

    if ray_upsample != 1:
        qsize = (img.shape[1] * ray_upsample, img.shape[0] * ray_upsample)
        # print(type(qsize))
        resize = lambda x : torch.from_numpy(cv2.resize(x.numpy(), qsize))


        OPA_FIX_MUL = 1024
        OPA_FIX_MOV = OPA_FIX_MUL - 1
        opa[opa < opa_thres] = -OPA_FIX_MOV
        opa = (opa + OPA_FIX_MOV) / OPA_FIX_MUL
        opa = resize(opa)
        opa = opa * OPA_FIX_MUL - OPA_FIX_MOV

        if debug:
            print("opa", opa[opa>0].min().item(), opa.max().item())
            print(((opa < opa_thres) * (opa > -OPA_FIX_MOV)).sum())
            show((opa < opa_thres) * (opa > -OPA_FIX_MOV))

            # return opa, None

        opa[opa < opa_thres] = 0

        img = torch.from_numpy(raw_img[:, :, :3][:, :, ::-1] / 255)
        img = resize(img)

    if for_eval:
        img = img * opa.unsqueeze(-1) + torch.tensor(BG_COLOR)
        orig_img = orig_img * orig_opa.unsqueeze(-1) + torch.tensor(BG_COLOR)

        # print(img.shape)

    if full_image:
        opa_thres = -math.inf

    dep = (opa >= opa_thres)
    orig_dep = (orig_opa >= opa_thres)
    assert orig_dep.shape == orig_img.shape[:2]
    assert dep.shape == img.shape[:2]
    # print(dep.shape, orig_dep.shape)

    if for_eval and BORDER_FIX_KER > 0:
        K = BORDER_FIX_KER
        H, W = dep.shape
        deps = []
        imgs = []

        resize = lambda x : torch.from_numpy(cv2.resize(x.numpy(), dep.shape[::-1]))
        tmp_img = resize(img)
        tmp_img[~dep] *= 0
        tmp_img[~dep] += torch.tensor(BG_COLOR).float()
        for a in range(K+1):
            for b in range(K+1):
                deps.append(dep[a:, b:][:H-K, :W-K])
                imgs.append(tmp_img[a:, b:][:H-K, :W-K])

        deps = torch.stack(deps, dim=-1).float().std(dim=-1)
        imgs = torch.stack(imgs, dim=-1).float().std(dim=-1).max(dim=-1).values
        bad_border = (deps > deps.mean() + 2 * deps.std()) & (imgs > imgs.mean() + BORDER_FIX_THRES * imgs.std())

        if debug:
            show(tmp_img)
            show(bad_border)

        HB, WB = bad_border.shape
        for a in range(K+1):
            for b in range(K+1):
                dep[a:, b:][:HB, :WB] &= ~bad_border

    def img_outlier(pts):
        W, H = orig_dep.shape
        pix = pts2pix_ns(cam, pts)
        pix = pix.long()
        ind = torch.arange(pts.shape[0], device='cuda')

        mask = (pix[:, 0] >= 0) & (pix[:, 1] >= 0) & (pix[:, 0] < H) & (pix[:, 1] < W)
        pix = pix[mask]
        ind = ind[mask]

        pix_ind = pix[:, 0] * W + pix[:, 1]
        inlier = orig_dep.cuda().reshape(-1)[pix_ind]
        ind = ind[inlier]

        if debug:
            outlier = ~inlier
            tmp_img = orig_dep * 0
            tmp_img.view(-1)[pix_ind[outlier]] = 1
            print("outliers:", outlier.sum())
            show(tmp_img)

        return ind

    img_outlier_fn = lambda pts: img_outlier(pts)


    ## DEBUG
    if debug:
        pass
        proj = pts2pix_ns(cam, pts)
        mask = (proj[:, 0] >= 0) & (proj[:, 1] >= 0) & (proj[:, 0] <= IMG_H) & (proj[:, 1] <= IMG_W)
        proj = proj[mask]
        pimg = torch.sparse_coo_tensor(indices=proj.long().T, values=pts_col[mask].mean(dim=-1), size=[IMG_W, IMG_H]).to_dense()
        pimg = pimg != 0
        show(pimg)

        pixels = torch.arange(IMG_W * IMG_H)
        restored = pts2pix_ns(cam, pix2pts_ns(cam, pixels)).long()
        restored = restored[:, 0] * IMG_W + restored[:, 1]
        print(pixels[:10], restored[:10])
        assert not (pixels != restored).any()


        print(pimg.shape, img.shape)
        # show(img)
        # show(dep)
        # show(img * dep[:,:,None])
        # show(img * (~dep[:,:,None]))
        print("opa:", opa.aminmax())
        show(opa)
        show(~dep)
        # show(img * opa.unsqueeze(-1))
        # show(img * pimg[:,:,None])
        # show(img * dep[:,:,None] * pimg[:,:,None])

        # print(orig_img.shape)
        # show(orig_img)
        # show(orig_dep)
        # show(orig_img * orig_dep[:,:,None])
        # show(orig_img * (~orig_dep[:,:,None]))

    if not for_eval:
        img[dep <= 0] -= 4
        orig_img[orig_dep <= 0] -= 4

        for d, k in zip([dep, orig_dep], [ray_upsample, 1]):
            tmp = d.clone()
            for k in range(1, 1 + IMAGE_TRAIN_BG_PAD*k):
                # BORDER PADDING
                d[k:, :] |= tmp[:-k, :]
                d[:-k, :] |= tmp[k:, :]
                d[:, k:] |= tmp[:, :-k]
                d[:, :-k] |= tmp[:, k:]

            if debug:
                show(tmp != d)


        if debug:
            print("border padding")
            show(img)
            show(orig_img)



    if for_eval or (not return_fn):
        # print("1")
        ans = img[dep].float()
        pix_used = torch.arange(img.shape[0] * img.shape[1]).reshape(img.shape[:2])[dep]
        line_o = cam_center.unsqueeze(0).expand(pix_used.shape[0], -1)
        line_d_fn = lambda : pix2ray_ns(cam, pix_used, scale=ray_upsample, rand=TRAIN_RANDOM_AUG and not for_eval)
        line_d = line_d_fn()
        data_fn = lambda : line_d_fn(), ans
    else:
        # print("2")
        ans_all = img
        opa_all = opa
        pix_all = torch.arange(img.shape[0] * img.shape[1])

        patch_dep = orig_dep
        patch_img = orig_img
        patch_size = IMAGE_PATCH_TRAIN * ray_upsample

        if IMAGE_PATCH_TRAIN > 1:
            a, b = orig_img.shape[:2][::-1]
            tmp = (a // IMAGE_PATCH_TRAIN, b // IMAGE_PATCH_TRAIN)
            resize = lambda x : torch.from_numpy(cv2.resize(x.numpy(), tmp, interpolation=cv2.INTER_NEAREST))
            patch_dep = resize(orig_dep.float()) > mask_thres

            if debug:
                print("patched")
                patch_img = resize(img)
                # show(patch_dep)
                # show(patch_img)
                # show(patch_dep[:,:,None]*patch_img)
                # show(~patch_dep[:,:,None]*patch_img)

        h, w = patch_dep.shape
        pix_all = pix_all.reshape(h, patch_size, w, patch_size)
        ans_all = ans_all.reshape(h, patch_size, w, patch_size, 3)
        opa_all = opa_all.reshape(h, patch_size, w, patch_size, 1)
        pix_all = pix_all.permute(0, 2, 1, 3).reshape(h, w, patch_size**2)[patch_dep]
        ans_all = ans_all.permute(0, 2, 1, 3, 4).reshape(h, w, patch_size**2, 3)[patch_dep]
        opa_all = opa_all.permute(0, 2, 1, 3, 4).reshape(h, w, patch_size**2, 1)[patch_dep]
        ans_all = torch.cat([ans_all, opa_all], dim=-1)
        line_o = cam_center.unsqueeze(0).expand(ans_all.shape[0], -1)

        if debug:
            print("check")
            cho = torch.randint(low=0, high=patch_size**2, size=[pix_all.shape[0], 1])
            pix = pix_all.gather(1, cho).squeeze(1)
            ans = ans_all.gather(1, cho.unsqueeze(-1).expand(-1, -1, 4)).squeeze(1)
            tmp = patch_img.clone()
            tmp[patch_dep] = ans[:, :3]
            show(tmp)

        # print(img.shape, ans_all.shape, ans_all.std(dim=1).sum())

        def generate_data():
            cho = torch.randint(low=0, high=patch_size**2, size=[pix_all.shape[0], 1])
            pix = pix_all.gather(1, cho).squeeze(1)
            ans = ans_all.gather(1, cho.unsqueeze(-1).expand(-1, -1, 4)).squeeze(1)
            line_d = pix2ray_ns(cam, pix, scale=ray_upsample, rand=TRAIN_RANDOM_AUG)
            return line_d, ans

        data_fn = lambda : generate_data()

    if debug:
        line_d, ans = data_fn()
        print("data_fn called", line_o.shape, line_d.shape, ans.shape)

    # print(line_o.shape, line_d.shape, ans.shape)

    ## DEBUG
    # if debug:
    #     mask = torch.randint(low=0, high=ans.shape[0], size=[50])
    #     plot(pts, conf=pts_conf, rgb=pts_col, pov=[1], cam_center=cam_center, rays=(line_o[mask], line_d[mask]))

    if for_eval:
        return line_o, line_d, ans, (dep, orig_dep, img.float(), orig_img.float())
    if return_fn:
        return line_o, data_fn, img_outlier_fn
    return line_o, line_d, ans

In [ ]:
line_o, data_fn, img_outlier_fn = read_data_ns(valid_config, 0, debug=True, full_image=False, ray_upsample=IMAGE_SCALE_TRAIN)
img_outlier_fn(pts.cuda())
img_outlier_fn(((pts - pts.mean(dim=0)) * 1.05 + pts.mean(dim=0)).cuda())

In [ ]:
class RayDataset(torch.utils.data.Dataset):
    def __init__(self, o, d=None, ans=None, fn=None):
        super().__init__()
        self.o = o
        self.d = d
        self.ans = ans
        self.fn = fn

    def generate_fn(self):
        if self.fn is not None:
            data = [f() for f in self.fn]
            self.d = torch.cat([d[0] for d in data], dim=0)
            self.ans = torch.cat([d[1] for d in data], dim=0)

    def __len__(self):
        return self.o.shape[0]

    def __getitem__(self, i):
        return self.o[i], self.d[i], self.ans[i]

    def loader(self, **kwargs):
        self.generate_fn()
        return torch.utils.data.DataLoader(self, **kwargs)

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, imgs):
        super().__init__()
        self.imgs = imgs

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, i):
        return self.imgs[i]

In [ ]:
if train_config is not None and not eval_only:
    train_o = []
    train_fn = []
    prune_outlier_fn = []

    for i in tqdm(range(len(train_config['frames']))):
        import gc
        gc.collect()
        o, fn, ofn = read_data_ns(train_config, i, ray_upsample=IMAGE_SCALE_TRAIN, return_fn=True)
        train_o.append(o)
        train_fn.append(fn)
        prune_outlier_fn.append(ofn)

    grow_o = train_o
    grow_fn = train_fn

    train_o = torch.cat(train_o * TRAIN_REPEAT, dim=0)
    train_fn = train_fn * TRAIN_REPEAT

    train_dataset = RayDataset(train_o, fn=train_fn)
    print(len(train_dataset))

In [ ]:
if train_config is not None and not eval_only:
    if grow_extra_config is not None:
        for i in tqdm(range(len(grow_extra_config['frames']))):
            o, fn = read_data_ns(grow_extra_config, i, ray_upsample=IMAGE_SCALE_TRAIN, return_fn=True)
            grow_o.append(o)
            grow_fn.append(fn)

    grow_o = torch.cat(grow_o * TRAIN_REPEAT, dim=0)
    grow_fn = grow_fn * TRAIN_REPEAT

    grow_dataset = RayDataset(grow_o, fn=grow_fn)
    print(len(grow_dataset))

In [ ]:
if train_config is not None and not eval_only:
    if prune_extra_config is not None:
        for i in tqdm(range(len(prune_extra_config['frames']))):
            o, fn, ofn = read_data_ns(prune_extra_config, i, ray_upsample=1, return_fn=True)
            prune_outlier_fn.append(ofn)

    len(prune_outlier_fn)

In [ ]:
valid_imgs = []

for i in tqdm(range(len(valid_config['frames']))):
    o, d, a, eval_info = read_data_ns(valid_config, i, for_eval=True, ray_upsample=IMAGE_SCALE)
    rayset = RayDataset(o, d, a)
    valid_imgs.append([rayset, eval_info])

valid_dataset = ImageDataset(valid_imgs)
print(len(valid_dataset))

In [ ]:
import gc
gc.collect()

In [ ]:
if train_config is not None and not eval_only:
    # preprocess cloud to remove image outliers
    ind = torch.arange(pts.shape[0], device='cuda')
    pts = pts.cuda()

    for ofn in tqdm(prune_outlier_fn):
        mask = ofn(pts)
        ind = ind[mask]
        pts = pts[mask]

    logging.info(f"maintain: remove {pts_embed.shape[0] - ind.shape[0]} image outliers")
    pts = pts.cpu()
    ind = ind.cpu()
    pts_embed = pts_embed[ind]
    pts_conf = pts_conf[ind]
    pts_col = pts_col[ind]

# Create Field

In [ ]:
from field.kd_field import KDField
from field.multi_field import MultiField

In [ ]:
MAX_LAYER = 22 if CLOUD_NAME == "cloud_gt" else 21

In [ ]:
def create_field(pts, pts_embed, pts_conf):
    return KDField(embed_dim=32, #32,

                   resume_immediately=eval_only,

                   max_pts_lim=2**21,
                   bnd_pad_fact=1, #2**0.5, # 1
                   spline_samples=8, num_support=96, num_support_nn=32,

                #    bnd_pad_ignore_bnd=True,

                   mesh_reg_weight=200,
                   warmup_mult=0.5,
                #    mesh_reg_weight=0,
                #    warmup_mult=1e-6,

                   # refnerf mode:
                   refnerf_mode=True,
                   manual_grad=False,
                   mlp_shapes=[16, 48, 128],

                   # classic mode:
                #    refnerf_mode=False,
                #    sh_deg=3,
                #    mlp_shapes=[32, 48, 72],

                   norm_mesh_param=NORM_MESH_PARAM,
                   reg_samples=1,
                   use_open3d_unit_len=True,
                   disable_conf_recenter=True,
                   adjust_height=True,
                   adaptive_support=True,
                   norm_mesh_remove_outlier=False,
                #    use_roughness=False,
                #    dropout=1/8, pts_dropout=0,
                   num_layer=MAX_LAYER, num_trees=1, pts=pts,
                   int_height_min=1, int_height_max=4, sample_height_max=5,
                   pts_embed=pts_embed, pts_conf=pts_conf, default_field_val=BG_COLOR,
                   train_single_tree=True).cuda()

In [ ]:
field = create_field(pts, pts_embed, pts_conf)

In [ ]:
field

In [ ]:
for name, param in field.state_dict().items():
    print(name, param.shape)

# Train

In [ ]:
img2mse = lambda x, y : torch.mean((x - y) ** 2)
mse2psnr = lambda x : -10. * torch.log(x) / torch.log(torch.Tensor([10.]).to(x.device))

In [ ]:
class Stat:
    def __init__(self):
        self.clear()

    def clear(self):
        self.correct = 0
        self.total = 0

    def add(self, cor, num=1, mean=True):
        if isinstance(cor, torch.Tensor):
            cor = cor.item()
        if isinstance(num, torch.Tensor):
            num = num.item()
        if mean:
            cor *= num
        self.correct += cor
        self.total += num

    def result(self, clear=True):
        ret = self.correct / max(self.total, 1e-5)
        if clear:
            self.clear()
        return ret

    def preview(self):
        return self.result(clear=False)

    def __str__(self):
        return "%.6lf" % self.result()

In [ ]:
class Prefix:
    def __init__(self, a, n=None, ratio=None):
        self.a = a

        if ratio is not None:
            assert n is None
            n = int(len(a) * ratio + 0.5)
        else:
            assert n is not None

        self.n = min(n, len(a))

    def __iter__(self):
        self.i = 0
        self.k = iter(self.a)
        return self

    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        self.i += 1
        return next(self.k)

    def __len__(self):
        return self.n

In [ ]:
def contour_rgb(pts):
    r = torch.tensor([1, 0, 0]).float().cuda()
    g = torch.tensor([0, 1, 0]).float().cuda()
    b = torch.tensor([0, 0, 1]).float().cuda()
    z = pts[:, 2].clone()
    min, max = z.aminmax()
    z = (z - min) / (max - min)
    z = z.unsqueeze(-1)
    c0 = (z - 0/2) * g + (1/2 - z) * b
    c1 = (z - 1/2) * r + (2/2 - z) * g
    c0 *= 2
    c1 *= 2
    rgb = torch.where(z.expand(-1, 3) < 0.5, c0, c1)
    return rgb

In [ ]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import sys
import lpips

lpips_alex = lpips.LPIPS(net='alex').cuda()
lpips_vgg = lpips.LPIPS(net='vgg').cuda()

def compute_metrics(x, y):
    with torch.no_grad():
        xx = x.permute(2, 0, 1).unsqueeze(0).cuda()
        yy = y.permute(2, 0, 1).unsqueeze(0).cuda()

        metrics = {
            'psnr': psnr(x.cpu().numpy(), y.cpu().numpy()),
            'ssim': ssim(x.cpu().numpy(), y.cpu().numpy(), win_size=11, multichannel=True),
            'lpips_alex': lpips_alex(xx, yy).item(),
            'lpips_vgg': lpips_vgg(xx, yy).item(),
            # 'fid': state.metrics["fid"],
        }

    return metrics

In [ ]:
def evaluate(field, valid_dataset, show_sample=False, show_all=False, show_predict_only=False,
             batch_size=4096, inpaint=False, norm_map=False, export_file=None):
    import gc

    gc.collect()
    field.deploy()
    stat_psnr = Stat()
    stat_ssim = Stat()
    stat_lpips_alex = Stat()
    stat_lpips_vgg = Stat()

    if show_sample is not False:
        if show_sample is True:
            show_sample = random.randint(0, len(valid_dataset) - 1)
    else:
        show_sample = None

    if norm_map:
        logging.debug(f"Norm Map:")

        pts = field.pts.cpu()
        norms = []
        with torch.no_grad():
            pts_embed = field.pts_embed()
            for i in tqdm(range(0, pts_embed.shape[0], 65536)):
                batch = pts_embed[i : i + 65536]
                norm = field.feed_pts_norm(field.feed_fea(batch))
                if field.enable_space_transform:
                    st = field.pts_space_transform[i : i + 65536]
                    st = st.reshape(-1, 3, 3)
                    norm = norm.unsqueeze(-2).matmul(st.transpose(-1, -2)).squeeze(-2)

                from field.kd_field import fix_norm_direction
                norm = fix_norm_direction(norm, -torch.tensor(DEFAULT_VIEW_DIR).float().cuda())
                norms.append(norm)
        norms = torch.cat(norms, dim=0).cpu()
        mask = torch.randperm(pts.shape[0])[:524288]
        plot(pts[mask], rgb=norms[mask]*0.5+0.5, dpi=64,
             save=f"./scratch/ns_{SCENE}_{CLOUD_NAME}_norm_pred.png")

    if export_file is not None:
        ckpt = []

    tbar = tqdm(valid_dataset, mininterval=5)
    tbar.set_description("Eval")
    for i, (img, eval_info) in enumerate(tbar):
        with logging_redirect_tqdm():
            predict = []
            loader = img.loader(batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=7)
            for j, (line_o, line_d, _) in enumerate(loader):
                if j % 100 == 0:
                    tbar.set_postfix_str(f"batch: {j+1}/{len(loader)}, psnr: {stat_psnr.preview():.4f}")
                line_o = line_o.cuda()
                line_d = line_d.cuda()
                with torch.no_grad():
                    pred, _, _ = field(line_o, line_d, sample_per_box=0)
                    pred[~field.line_intersect] -= 100000
                    predict.append(pred.cpu())

            scaled_mask, orig_mask, scaled_img, orig_img = eval_info
            predict = torch.cat(predict, dim=0)

            scaled_img[scaled_mask] = predict
            predict = scaled_img

            if inpaint and (predict < 0).any():
                zero_mask = (predict.min(dim=-1).values < 0).to(torch.uint8)
                predict = torch.from_numpy(cv2.inpaint((predict * 255).to(torch.uint8).numpy(),
                                                    zero_mask.numpy(), 3 * IMAGE_SCALE, cv.INPAINT_TELEA)) / 255
                if zero_mask.sum().item() > 100:
                    logging.debug(f"Inpainted {zero_mask.sum().item()} pixels")

            if IMAGE_SCALE != 1:
                qsize = (orig_img.shape[1], orig_img.shape[0])
                resize = lambda x : torch.from_numpy(cv2.resize(x.numpy(), qsize,
                                                                interpolation=cv2.INTER_NEAREST))
                predict = resize(predict)

            predict[~orig_mask] = orig_img[~orig_mask]
            target = orig_img

            predict = predict.clamp(min=0, max=1)
            # make sure both are 8bit
            # predict = (predict * 255).round() / 255
            # target = (target * 255).round() / 255

            if export_file is not None:
                ckpt.append([predict.cpu(), target.cpu()])

            if show_all:
                a, b = predict, target
                logging.debug(f"Image #{i}")
                logging.debug(f"Predict:")
                show(a.numpy(),
                    save=f"./scratch/ns_{SCENE}_{CLOUD_NAME}_valid{i}_pred.png")

                if not show_predict_only:
                    logging.debug(f"Target:")
                    show(b.numpy(),
                        save=f"./scratch/ns_{SCENE}_{CLOUD_NAME}_valid{i}_target.png")
                    logging.debug(f"Diff:")
                    show((a - b).abs().mean(dim=-1).clamp(min=0, max=1).numpy(),
                        save=f"./scratch/ns_{SCENE}_{CLOUD_NAME}_valid{i}_diff.png")

            elif i == show_sample:
                sample = (predict, target)

            # Eval with orig_mask
            if EVAL_MASK_ONLY:
                predict = preidct[orig_mask]
                target = target[orig_mask]

            # stat_psnr.add(mse2psnr(img2mse(predict, target)))
            metrics = compute_metrics(predict, target)
            stat_psnr.add(metrics['psnr'])
            stat_ssim.add(metrics['ssim'])
            stat_lpips_alex.add(metrics['lpips_alex'])
            stat_lpips_vgg.add(metrics['lpips_vgg'])

            if show_all:
                logging.debug(f"PSNR: {mse2psnr(img2mse(predict, target)).item():.6f}")

            if i % 10 == 0:
                logging.info(f"Eval {i}/{len(tbar)}: psnr: {stat_psnr.preview():.4f}")

    field.undeploy()

    if export_file is not None:
        logging.info(f"Save to {export_file}")
        torch.save(ckpt, export_file)

    if (not show_all) and show_sample is not None:
        a, b = sample
        logging.debug(f"Sample image #{show_sample}")
        logging.debug(f"Predict:")
        show(a.numpy(),
             save=f"./scratch/ns_{SCENE}_{CLOUD_NAME}_valid{show_sample}_pred.png")
        if not show_predict_only:
            logging.debug(f"Target:")
            show(b.numpy(),
                save=f"./scratch/ns_{SCENE}_{CLOUD_NAME}_valid{show_sample}_target.png")
            logging.debug(f"Diff:")
            show((a - b).abs().mean(dim=-1).clamp(min=0, max=1).numpy(),
                save=f"./scratch/ns_{SCENE}_{CLOUD_NAME}_valid{show_sample}_diff.png")

    gc.collect()

    logging.info(f"psnr: {stat_psnr.preview()}")
    logging.info(f"ssim: {stat_ssim.preview()}")
    logging.info(f"lpips_alex: {stat_lpips_alex.preview()}")
    logging.info(f"lpips_vgg: {stat_lpips_vgg.preview()}")

    return stat_psnr

In [ ]:
def train(field, exp_name="field", OUTPUT="./scratch/", lr=5e-3, min_lr=1e-6, valid_epoch=1, eloss_coef=0,
          num_epoch=1000000, start_epoch=0, batch_size=4096, grow_batch_size=8192, eval_batch_size=4096, resume_ckpt=None,
          sch_T=10, grow_sch_T_fact=1, grow_epoch_fact=1, rebuild_tree_fact=2, conf_reg_coef=0.0003,
          sch_start_0=False, first_grow_epoch=10, last_grow_epoch=100):
    import math
    import gc
    logging.info(f"Train {exp_name} OUTPUT = {OUTPUT}")



    col_encode_rep = 0

    print_epoch = 1
    show_epoch = valid_epoch

    save_epoch = 1

    rebuild_tree_epoch = rebuild_tree_fact * sch_T
    grow_sch_T = int(grow_sch_T_fact * sch_T + 0.5)
    grow_epoch = int(grow_epoch_fact * grow_sch_T + 0.5)

    sample_per_box = 4
    do_grow = False

    best_eval_res = -1

    def get_opt_sch(field, T):
        logging.info(f"init opt_sch T = {T}")
        opt = torch.optim.Adam(field.parameters(), lr=lr)
        # sch = torch.optim.lr_scheduler.ExponentialLR(opt, 0.5)
        # sch_step_epoch = 10
        sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=T, eta_min=min_lr)
        return opt, sch
    T = grow_sch_T if (first_grow_epoch <= start_epoch < last_grow_epoch) else sch_T
    opt, sch = get_opt_sch(field, T)
    sch_step_epoch = 1
    if sch_start_0:
        opt.zero_grad()
        opt.step()
        for i in range(sch_T):
            sch.step()
        logging.info(f"start_0 lr = {sch.get_last_lr()[0]}")

    if resume_ckpt:
        resume_ckpt, ckpt = resume_ckpt
        logging.info(f"ckpt loaded from {resume_ckpt}")
        opt.load_state_dict(ckpt['opt'])
        sch.load_state_dict(ckpt['sch'])
        best_eval_res = ckpt['best_eval_res']
        del ckpt
        gc.collect()


    def hyperparam(name, var_map=locals() | globals()):
        val = var_map[name]
        logging.info(f"Hyperparam {name} = {val}")

    if True:
        logging.info("Dataset hyperparams:")
        hyperparam("SCENE")
        hyperparam("DATASET")
        hyperparam("CLOUD_NAME")
        hyperparam("LIGHT_IND")
        hyperparam("CLOUD_SCALE")
        hyperparam("IMAGE_SCALE")
        hyperparam("IMAGE_SCALE_TRAIN")
        hyperparam("IMAGE_PATCH_TRAIN")
        hyperparam("TRAIN_RANDOM_AUG")
        hyperparam("TRAIN_REPEAT")
        hyperparam("NUM_PTS")
        hyperparam("IGNORE_CONF")
        hyperparam("IGNORE_EMBED")
        hyperparam("BORDER_FIX_KER")
        hyperparam("BORDER_FIX_THRES")
        hyperparam("EVAL_MASK_ONLY")
        hyperparam("BG_COLOR")
        hyperparam("IMAGE_TRAIN_BG_PAD")

        logging.info("Model hyperparams:")
        hyperparam("MAX_LAYER")
        hyperparam("lr")
        hyperparam("sch_T")
        hyperparam("grow_sch_T")
        hyperparam("start_epoch")
        hyperparam("batch_size")
        hyperparam("grow_batch_size")
        hyperparam("eval_batch_size")
        hyperparam("eloss_coef")
        hyperparam("col_encode_rep")
        hyperparam("sch_step_epoch")
        hyperparam("grow_epoch")
        hyperparam("rebuild_tree_epoch")
        hyperparam("sample_per_box")
        hyperparam("sch_start_0")
        hyperparam("first_grow_epoch")
        hyperparam("last_grow_epoch")
        hyperparam("best_eval_res")
        hyperparam("conf_reg_coef")

    def debug_plot(name, pts, rgb):
        logging.debug(name + ":")
        if pts is None:
            logging.debug("No points")
            return
        pts = pts.cpu()
        rgb = torch.zeros_like(pts) + rgb.cpu()
        rgb = rgb.clamp(min=0, max=1)
        mask = torch.randperm(pts.shape[0])[:524288]
        plot(pts[mask], rgb=rgb[mask], save=f"./scratch/ns_{SCENE}_{CLOUD_NAME}_debug_{name}.png")

    if (resume_ckpt is None) or ("postgrow" not in resume_ckpt):
        field.fix_norm_mesh_dir(
            train_dataset.loader(batch_size=batch_size*2, shuffle=True, pin_memory=True, num_workers=7))
        new_pts = field.pts
        new_norm = field.pts_norm_mesh
        mask = new_norm.norm(dim=-1) > 0.5
        debug_plot("initial_norm_raw", new_pts[mask], new_norm[mask]*0.5+0.5)
    else:
        logging.debug("postgrow loaded, skip initial fix dir")


    stat_ploss = Stat()
    stat_eloss = Stat()
    stat_psnr = Stat()

    if eval_batch_size is None:
        eval_batch_size = batch_size * 2

    def create_ckpt():
        return {
            'epoch': epoch,
            'field': field.state_dict(),
            'opt': opt.state_dict(),
            'sch': sch.state_dict(),
            'best_eval_res': best_eval_res,
        }

    if start_epoch > 1:
        train_loader = train_dataset.loader(batch_size=batch_size)
        n = len(train_loader)
        for i in range(1, start_epoch):
            epoch = i
            logging.info(f"Skip warming up epoch #{i}")
            if first_grow_epoch <= epoch <= last_grow_epoch and epoch % grow_epoch == 0:
                field.warmup_coef = 1
                logging.info(f"reset warmup coef to {field.warmup_coef}")
            for _ in range(n):
                field.warmup_step(n)

    for epoch in range(start_epoch, num_epoch + 1):
        gc.collect()

        if epoch > 0:

            if first_grow_epoch <= epoch <= last_grow_epoch and epoch % grow_epoch == first_grow_epoch % grow_epoch:
                logging.info("grow epoch")

                if epoch == start_epoch and (resume_ckpt is not None) and ("postgrow" in resume_ckpt):
                    logging.info("loaded postgrow ckpt, skipping")
                    grow_pts = prune_pts = None
                    new_pts = field.pts
                    new_norm = field.pts_norm_mesh
                else:
                    ckpt = create_ckpt()
                    torch.save(ckpt, f"{OUTPUT}/{exp_name}_train_pregrow.ckpt")
                    logging.debug(f"Saved pregrow")
                    grow_loader = grow_dataset.loader(batch_size=grow_batch_size, shuffle=True, pin_memory=True, num_workers=7)
                    # grow_loader = Prefix(grow_loader, len(grow_loader) // 3)
                    grow_pts, prune_pts, new_pts, new_norm = field.prune_and_grow(grow_loader, prune_outlier_fn)

                field.fix_norm_mesh_dir(
                    train_dataset.loader(batch_size=batch_size*2, shuffle=True, pin_memory=True, num_workers=7))
                new_norm = field.pts_norm_mesh

                debug_plot(f"grow_points_{epoch}ep", grow_pts, contour_rgb(grow_pts))
                debug_plot(f"prune_points_{epoch}ep", prune_pts, contour_rgb(prune_pts))

                from field.kd_field import fix_norm_direction
                mask = new_norm.norm(dim=-1) > 0.5
                debug_plot(f"new_norm_raw_{epoch}ep", new_pts[mask], new_norm[mask]*0.5+0.5)
                new_norm = fix_norm_direction(new_norm, -torch.tensor(DEFAULT_VIEW_DIR).float().cuda())
                debug_plot(f"new_norm_{epoch}ep", new_pts[mask], new_norm[mask]*0.5+0.5)

                # if epoch == last_grow_epoch:
                #     field.enable_st()

                ckpt = create_ckpt()
                torch.save(ckpt, f"{OUTPUT}/{exp_name}_train_postgrow.ckpt")
                logging.debug(f"Saved postgrow")

                del opt
                del sch
                gc.collect()
                T = grow_sch_T if (first_grow_epoch <= epoch < last_grow_epoch) else sch_T
                opt, sch = get_opt_sch(field, T)
                field.warmup_coef = 1
                logging.info(f"reset warmup coef to {field.warmup_coef}")

            else:
                # prune_mask = field.pts_prune()
                # if prune_mask.sum().item() > 0:
                #     prune_pts = field.pts[prune_mask]
                # else:
                #     prune_mask = None
                #     prune_pts = None

                # debug_plot("pseudo-prune points", prune_pts, torch.tensor([1., 0., 0.]))
                # field.pseudo_prune(prune_mask)

                do_grow = False

            samples = []
            num_samples = 0

            train_loader = train_dataset.loader(batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=7)
            tbar = tqdm(train_loader, mininterval=5)
            tbar.set_description(f"Train #{epoch}")

            field.undeploy()

            for index, (line_o, line_d, target) in enumerate(tbar):
                assert field.training
                line_o = line_o.cuda()
                line_d = line_d.cuda()
                target = target.cuda()
                target, density_target = target.split([3, 1], dim=-1)

                with logging_redirect_tqdm():
                    pred_img, eloss, smp = field(line_o, line_d, sample_per_box=0)
                    predict = field.field
                    density_predict = field.density

                ins = field.line_intersect
                predict = predict[ins]
                target = target[ins].float()
                background = target.min(dim=-1).values < -0.5
                target[background] += 4

                density_predict = density_predict[ins]
                density_target = density_target[ins].float()

                # density_loss = img2mse(density_predict, density_target)

                # predict[background] = predict[background] + (1 - density_predict[background]) * (
                #                 target[background] - torch.tensor(BG_COLOR).float().cuda())

                pred_img = pred_img[ins]
                targ_img = target * density_target + field.default_field_val * (1 - density_target)
                img_loss = img2mse(pred_img, targ_img)
                field_loss = (predict - target.clamp(min=0, max=1)).pow(2).mean(dim=-1)
                density_loss = (density_predict - density_target.clamp(min=0, max=1)).pow(2).reshape(-1)

                sep_loss = field_loss * density_target.reshape(-1) + density_loss * (1 - density_target.reshape(-1)) + 0.1 * density_loss
                if epoch < last_grow_epoch:
                    ploss = img_loss + sep_loss
                else:
                    ploss = img_loss + 0.05 * sep_loss

                ploss = ploss.mean()
                eloss = eloss + (conf_reg_coef / eloss_coef) * field.conf_reg_loss
                loss =  ploss + eloss_coef * eloss

                assert not ploss.isnan().any()
                assert not eloss.isnan().any()

                opt.zero_grad()
                loss.backward()
                # torch.nn.utils.clip_grad_norm_(field.parameters(), 1e-3, norm_type='inf', error_if_nonfinite=True)
                opt.step()
                last_lr = sch.get_last_lr()[0]
                field.warmup_step(len(train_loader))

                bs = ins.sum().item()
                with torch.no_grad():
                    stat_ploss.add(ploss, bs)
                    stat_eloss.add(eloss, bs)
                    stat_psnr.add(mse2psnr(img_loss), bs)

                if index % 25 == 0:
                    with torch.no_grad():
                        conf = field.pts_conf()
                        bad = (conf < 0.1).sum().item()
                        logstr = f"-{bad}, +{num_samples}, loss: {stat_ploss.preview():.4f} {stat_eloss.preview():.4f}, psnr: {stat_psnr.preview():.4f}"
                    tbar.set_postfix_str(logstr)
                    if index % 500 == 0:
                        with logging_redirect_tqdm():
                            logging.info(f"Train #{epoch} {index}/{len(tbar)} loss = {stat_ploss.preview():.4f} {stat_eloss.preview():.4f} psnr = {stat_psnr.preview():.4f}")


            if epoch % sch_step_epoch == 0:
                sch.step()

            if epoch % print_epoch == 0:
                with torch.no_grad():
                    conf = field.pts_conf()
                    bad = (conf < 0.1).sum().item()

                logging.info(f"Train #{epoch} @{last_lr:.2e} "
                    + f"loss = {stat_ploss} {stat_eloss.result():.4f} -{bad} +{num_samples} "
                    + f"psnr = {stat_psnr} "
                )

            # DEBUG ASSERT
            assert not loss.isnan().any()
            field.assert_no_nan(force=True)

        if epoch % save_epoch == 0:

            ckpt = create_ckpt()
            torch.save(ckpt, f"{OUTPUT}/{exp_name}_train_{epoch}ep.ckpt")
            logging.info(f"Saved #{epoch}")

        if epoch % valid_epoch == 0:
            eval_res_stat = evaluate(field, valid_dataset, norm_map=True,
                                     show_sample=epoch <= 5 or epoch % show_epoch == 0,
                                     batch_size=int(eval_batch_size * (
                                         0.75 if field.pts.shape[0] > 0.5 * field.max_pts_lim else 1) + 0.5),
                                     export_file=f"{OUTPUT}/valid_images_{exp_name}_{epoch}ep.pth")
            eval_res = eval_res_stat.preview()
            updated = ""
            if eval_res > best_eval_res:
                best_eval_res = eval_res
                updated = "updated"
                ckpt = create_ckpt()
                torch.save(ckpt, f"{OUTPUT}/{exp_name}_train_best.ckpt")
            logging.info(f"Valid #{epoch} psnr = {eval_res_stat} {updated}")

            if epoch % save_epoch == 0:
                ckpt = create_ckpt()
                torch.save(ckpt, f"{OUTPUT}/{exp_name}_train_{epoch}ep.ckpt")
                logging.info(f"Saved #{epoch} (after valid)")

        if epoch > 0 and epoch % rebuild_tree_epoch == 0:
            field.rebuild_trees()
            logging.debug("Trees rebuilt")

            del opt
            del sch
            gc.collect()
            T = grow_sch_T if (first_grow_epoch <= epoch < last_grow_epoch) else sch_T
            opt, sch = get_opt_sch(field, T)

        ckpt = create_ckpt()
        torch.save(ckpt, f"{OUTPUT}/{exp_name}_train_latest.ckpt")
        logging.debug(f"Saved latest")

In [ ]:
start_epoch = 1
resume_ckpt = None

# Loop

In [ ]:
LABEL = "grow"
PREV_STAGE_LABEL = "grow"

In [ ]:
# ckpt_file = f"./scratch/field_ns_{SCENE}_{CLOUD_NAME}_layer{MAX_LAYER}_{LABEL}_train_30ep.ckpt"
# start_epoch, ckpt = field.resume_ckpt(ckpt_file)
# resume_ckpt = (ckpt_file, ckpt)
# if ("pregrow" in ckpt_file) or ("postgrow" in ckpt_file):  start_epoch -= 1
# evaluate(field, [], norm_map=True)

In [ ]:
start_epoch

In [ ]:
if not eval_only:
    # grow phase 1
    if start_epoch <= 30:
        train(field, f"field_ns_{SCENE}_{CLOUD_NAME}_layer{MAX_LAYER}_{LABEL}",
            lr=2e-3, eloss_coef=1e-5,
            conf_reg_coef=0.0002*TRAIN_REPEAT,
            valid_epoch=2,
            num_epoch=30,
            sch_T=20, grow_sch_T_fact=1, grow_epoch_fact=1/20, rebuild_tree_fact=4,
            start_epoch=start_epoch,
            resume_ckpt=resume_ckpt,
            first_grow_epoch=4, last_grow_epoch=12,
            batch_size=4096*TRAIN_REPEAT, grow_batch_size=4096*TRAIN_REPEAT, eval_batch_size=4096)

        start_epoch = 30 + 1
        resume_ckpt = None

    # grow phase 2
    train(field, f"field_ns_{SCENE}_{CLOUD_NAME}_layer{MAX_LAYER}_{LABEL}",
        lr=2e-3, eloss_coef=1e-5/2,
        conf_reg_coef=0.0002/2*TRAIN_REPEAT,
        valid_epoch=2,
        num_epoch=70,
        sch_T=40, grow_sch_T_fact=1, grow_epoch_fact=1/2, rebuild_tree_fact=4,
        start_epoch=start_epoch,
        resume_ckpt=resume_ckpt,
        first_grow_epoch=31, last_grow_epoch=31,
        batch_size=4096*TRAIN_REPEAT, grow_batch_size=4096*TRAIN_REPEAT, eval_batch_size=4096)

# Eval

## Defs

In [ ]:
import math
from math import cos, sin

In [ ]:
train_o = []
train_fn = []

for i in tqdm(range(len(train_config['frames']))):
    import gc
    gc.collect()
    o, fn, ofn = read_data_ns(train_config, i, ray_upsample=1, full_image=True, return_fn=True)
    train_o.append(o)
    train_fn.append(fn)

train_o = torch.cat(train_o * TRAIN_REPEAT, dim=0)
train_fn = train_fn * TRAIN_REPEAT

train_dataset = train_full_dataset = RayDataset(train_o, fn=train_fn)
print(len(train_dataset))

In [ ]:
valid_full_imgs = []

for i in tqdm(range(len(valid_config['frames']))):
    o, d, a, eval_info = read_data_ns(valid_config, i, for_eval=True, full_image=True, ray_upsample=IMAGE_SCALE)
    rayset = RayDataset(o, d, a)
    valid_full_imgs.append([rayset, eval_info])

valid_dataset = valid_full_dataset = ImageDataset(valid_full_imgs)
print(len(valid_dataset))

valid_subset = [valid_full_dataset[i] for i in range(0, 200, 20)]

In [ ]:
def plot_mesh_norm(fix=True):
    def debug_plot(name, pts, rgb):
        logging.debug(name + ":")
        if pts is None:
            logging.debug("No points")
            return
        pts = pts.cpu()
        rgb = torch.zeros_like(pts) + rgb.cpu()
        rgb = rgb.clamp(min=0, max=1)
        mask = torch.randperm(pts.shape[0])[:524288]
        plot(pts[mask], rgb=rgb[mask])

    from field.kd_field import fix_norm_direction
    new_norm = field.pts_norm_mesh
    if fix:
        new_norm = fix_norm_direction(new_norm, -torch.tensor(DEFAULT_VIEW_DIR).float().cuda())
    else:
        new_norm = new_norm
    mask = new_norm.norm(dim=-1) > 0.5
    debug_plot("new norm", field.pts[mask], new_norm[mask]*0.5+0.5)

In [ ]:
assert not field.enable_space_transform

In [ ]:
# field.editting = True
# field.orig_pts = field.pts.clone()
# field.orig_unit_len = field.unit_len
# field.orig_pts_norm_mesh = field.pts_norm_mesh.clone()
# field.fixed_norm_mesh_dir = True
# field.calculate_norm_mesh(remove_outlier=False)

# M = field.estimate_initial_st()
# M = M.reshape(-1, 3, 3)
# M

## Deform Init


In [ ]:
try:
    deform_pts = torch.load(f"./datasets/Deform/{DEFORM}.{VER}.pth")['pts'].float().cuda()
except:
    import numpy as np
    deform_pts = np.load(f"./datasets/Deform/{DEFORM}.{VER}.npy")
    deform_pts = torch.from_numpy(np.array(deform_pts)).float().cuda()

plot(deform_pts, rgb=contour_rgb(deform_pts))
deform_pts.shape

In [ ]:
ckpt_file = f"./datasets/Models/scene_edit/{SCENE}.{VER}.ckpt"
start_epoch, ckpt = field.resume_ckpt(ckpt_file)

assert field.pts.shape == deform_pts.shape

In [ ]:
field.fix_norm_mesh_dir(
    Prefix(train_dataset.loader(batch_size=8192, shuffle=True, pin_memory=True, num_workers=7), ratio=0.1))
plot_mesh_norm(fix=False)

In [ ]:
DATASET = f"./datasets/Deform/{DEFORM}_data"
train_config = json.load(open(f"{DATASET}/views_train/transforms.json"))
grow_extra_config = None #json.load(open(f"{DATASET}/transforms_val.json"))
prune_extra_config = None
valid_config = json.load(open(f"{DATASET}/views_test/transforms.json"))

In [ ]:
train_o = []
train_fn = []

for i in tqdm(range(len(train_config['frames']))):
    import gc
    gc.collect()
    o, fn, ofn = read_data_ns(train_config, i, ray_upsample=1, full_image=True, return_fn=True)
    train_o.append(o)
    train_fn.append(fn)

train_o = torch.cat(train_o * TRAIN_REPEAT, dim=0)
train_fn = train_fn * TRAIN_REPEAT

train_dataset = RayDataset(train_o, fn=train_fn)
print(len(train_dataset))

In [ ]:
field.init_pts(deform_pts, editting=True)
field.enable_st()

In [ ]:
field.fix_norm_mesh_dir(
    Prefix(train_dataset.loader(batch_size=8192, shuffle=True, pin_memory=True, num_workers=7), ratio=0.1))
plot_mesh_norm(fix=False)

In [ ]:
DEFORM_CKPT = f"./scratch/deform_{DEFORM}_init.ckpt"
torch.save(field.state_dict(), DEFORM_CKPT)

## Deform Full

In [ ]:
field.load_state_dict(torch.load(DEFORM_CKPT))
field.naive_plot_mode = False
field.init_space_transform = True

In [ ]:
train_o = []
train_fn = []

for i in tqdm(range(len(train_config['frames']))):
    import gc
    gc.collect()
    o, fn, ofn = read_data_ns(train_config, i, ray_upsample=1, full_image=True, return_fn=True)
    train_o.append(o)
    train_fn.append(fn)

train_o = torch.cat(train_o * TRAIN_REPEAT, dim=0)
train_fn = train_fn * TRAIN_REPEAT

train_dataset = RayDataset(train_o, fn=train_fn)
print(len(train_dataset))

In [ ]:
valid_full_imgs = []

for i in tqdm(range(len(valid_config['frames']))):
    o, d, a, eval_info = read_data_ns(valid_config, i, for_eval=True, full_image=True, ray_upsample=IMAGE_SCALE)
    rayset = RayDataset(o, d, a)
    valid_full_imgs.append([rayset, eval_info])

valid_dataset = valid_full_dataset = ImageDataset(valid_full_imgs)
print(len(valid_dataset))

valid_subset = [valid_full_dataset[i] for i in range(0, len(valid_dataset), 10)]

In [ ]:
field.enable_st(reinit=True)
field.train_st(
    Prefix(train_dataset.loader(batch_size=8192, shuffle=True, pin_memory=True, num_workers=7), ratio=0.1), num_epoch=1)

In [ ]:
print(evaluate(field, valid_subset, inpaint=False, show_all=True, show_predict_only=False, norm_map=False))

In [ ]:
train_o = []
train_fn = []

for i in tqdm(range(len(train_config['frames']))):
    import gc
    gc.collect()
    o, fn, ofn = read_data_ns(train_config, i, ray_upsample=1, full_image=False, return_fn=True)
    train_o.append(o)
    train_fn.append(fn)

train_o = torch.cat(train_o * TRAIN_REPEAT, dim=0)
train_fn = train_fn * TRAIN_REPEAT

train_dataset = RayDataset(train_o, fn=train_fn)
print(len(train_dataset))

In [ ]:
field.warmup_coef = 1e-4
field.warmup_mult = 1e-4
train(field, f"field_ns_deform_{DEFORM}",
    lr=2e-3, eloss_coef=1e-5,
    conf_reg_coef=0.0002,
    valid_epoch=1,
    num_epoch=1,
    sch_T=6, grow_sch_T_fact=1, grow_epoch_fact=1/6, rebuild_tree_fact=4,
    start_epoch=0,
    resume_ckpt=None,
    first_grow_epoch=0, last_grow_epoch=0,
    # batch_size=3072, grow_batch_size=3072, eval_batch_size=4096)
    batch_size=4096, grow_batch_size=4096, eval_batch_size=4096)

In [ ]:
print(evaluate(field, valid_subset, inpaint=False, show_all=True, show_predict_only=False, norm_map=False))

## Deform Naive


In [ ]:
field.load_state_dict(torch.load(DEFORM_CKPT))
field.naive_plot_mode = True
field.init_space_transform = False

In [ ]:
train_o = []
train_fn = []

for i in tqdm(range(len(train_config['frames']))):
    import gc
    gc.collect()
    o, fn, ofn = read_data_ns(train_config, i, ray_upsample=1, full_image=True, return_fn=True)
    train_o.append(o)
    train_fn.append(fn)

train_o = torch.cat(train_o * TRAIN_REPEAT, dim=0)
train_fn = train_fn * TRAIN_REPEAT

train_dataset = RayDataset(train_o, fn=train_fn)
print(len(train_dataset))

In [ ]:
valid_full_imgs = []

for i in tqdm(range(len(valid_config['frames']))):
    o, d, a, eval_info = read_data_ns(valid_config, i, for_eval=True, full_image=True, ray_upsample=IMAGE_SCALE)
    rayset = RayDataset(o, d, a)
    valid_full_imgs.append([rayset, eval_info])

valid_dataset = valid_full_dataset = ImageDataset(valid_full_imgs)
print(len(valid_dataset))

valid_subset = [valid_full_dataset[i] for i in range(0, len(valid_dataset), 10)]

In [ ]:
field.enable_st(reinit=True)
field.train_st(
    Prefix(train_dataset.loader(batch_size=8192, shuffle=True, pin_memory=True, num_workers=7), ratio=0.1), num_epoch=1)

In [ ]:
print(evaluate(field, valid_subset, inpaint=False, show_all=True, show_predict_only=False, norm_map=False))

In [ ]:
train_o = []
train_fn = []

for i in tqdm(range(len(train_config['frames']))):
    import gc
    gc.collect()
    o, fn, ofn = read_data_ns(train_config, i, ray_upsample=1, full_image=False, return_fn=True)
    train_o.append(o)
    train_fn.append(fn)

train_o = torch.cat(train_o * TRAIN_REPEAT, dim=0)
train_fn = train_fn * TRAIN_REPEAT

train_dataset = RayDataset(train_o, fn=train_fn)
print(len(train_dataset))

In [ ]:
field.warmup_coef = 1e-4
field.warmup_mult = 1e-4
train(field, f"field_ns_deform_{DEFORM}_naive",
    lr=2e-3, eloss_coef=1e-5,
    conf_reg_coef=0.0002,
    valid_epoch=1,
    num_epoch=1,
    sch_T=6, grow_sch_T_fact=1, grow_epoch_fact=1/6, rebuild_tree_fact=4,
    start_epoch=0,
    resume_ckpt=None,
    first_grow_epoch=0, last_grow_epoch=0,
    # batch_size=3072, grow_batch_size=3072, eval_batch_size=4096)
    batch_size=4096, grow_batch_size=4096, eval_batch_size=4096)

In [ ]:
print(evaluate(field, valid_subset, inpaint=False, show_all=True, show_predict_only=False, norm_map=False))

## Deform NoInitSt


In [ ]:
field.load_state_dict(torch.load(DEFORM_CKPT))
field.naive_plot_mode = False
field.init_space_transform = False

In [ ]:
train_o = []
train_fn = []

for i in tqdm(range(len(train_config['frames']))):
    import gc
    gc.collect()
    o, fn, ofn = read_data_ns(train_config, i, ray_upsample=1, full_image=True, return_fn=True)
    train_o.append(o)
    train_fn.append(fn)

train_o = torch.cat(train_o * TRAIN_REPEAT, dim=0)
train_fn = train_fn * TRAIN_REPEAT

train_dataset = RayDataset(train_o, fn=train_fn)
print(len(train_dataset))

In [ ]:
valid_full_imgs = []

for i in tqdm(range(len(valid_config['frames']))):
    o, d, a, eval_info = read_data_ns(valid_config, i, for_eval=True, full_image=True, ray_upsample=IMAGE_SCALE)
    rayset = RayDataset(o, d, a)
    valid_full_imgs.append([rayset, eval_info])

valid_dataset = valid_full_dataset = ImageDataset(valid_full_imgs)
print(len(valid_dataset))

valid_subset = [valid_full_dataset[i] for i in range(0, len(valid_dataset), 10)]

In [ ]:
field.enable_st(reinit=True)
field.train_st(
    Prefix(train_dataset.loader(batch_size=8192, shuffle=True, pin_memory=True, num_workers=7), ratio=0.1), num_epoch=1)

In [ ]:
print(evaluate(field, valid_subset, inpaint=False, show_all=True, show_predict_only=False, norm_map=False))

In [ ]:
train_o = []
train_fn = []

for i in tqdm(range(len(train_config['frames']))):
    import gc
    gc.collect()
    o, fn, ofn = read_data_ns(train_config, i, ray_upsample=1, full_image=False, return_fn=True)
    train_o.append(o)
    train_fn.append(fn)

train_o = torch.cat(train_o * TRAIN_REPEAT, dim=0)
train_fn = train_fn * TRAIN_REPEAT

train_dataset = RayDataset(train_o, fn=train_fn)
print(len(train_dataset))

In [ ]:
field.warmup_coef = 1e-4
field.warmup_mult = 1e-4
train(field, f"field_ns_deform_{DEFORM}_noinitst",
    lr=2e-3, eloss_coef=1e-5,
    conf_reg_coef=0.0002,
    valid_epoch=1,
    num_epoch=1,
    sch_T=6, grow_sch_T_fact=1, grow_epoch_fact=1/6, rebuild_tree_fact=4,
    start_epoch=0,
    resume_ckpt=None,
    first_grow_epoch=0, last_grow_epoch=0,
    # batch_size=3072, grow_batch_size=3072, eval_batch_size=4096)
    batch_size=4096, grow_batch_size=4096, eval_batch_size=4096)

In [ ]:
print(evaluate(field, valid_subset, inpaint=False, show_all=True, show_predict_only=False, norm_map=False))

# Archive

In [ ]:
assert False

In [ ]:
oz = -math.pi / 2
oz = torch.tensor([
    [cos(oz), -sin(oz), 0],
    [sin(oz), cos(oz), 0],
    [0, 0, 1]
]).float().cuda()

oz